In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
from springstone.data import create_df_for_prophet, create_train_test, get_data
from springstone.params import MODEL_TYPE, TICKERS, PROPHET_COLUMN
from springstone.utils import prophet_non_business_days, prophet_preprocessing
from springstone.trainer import Trainer
from springstone.predict import evaluate_model
import joblib
from tensorflow.keras.models import load_model
from springstone.data import get_data
import numpy as np

2022-03-17 15:28:24.205119: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-17 15:28:24.205171: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2022-03-17 15:28:29,444 loading file /home/blueoverseas/.flair/models/sentiment-en-mix-distillbert_4.pt


In [3]:
ticker = 'AAPL'

In [4]:
df = get_data(ticker, end='2022-03-12')
preprocessor = joblib.load(f'../preprocessor_{ticker}.joblib')

[*********************100%***********************]  1 of 1 completed


In [5]:
df_nbd = prophet_non_business_days(df)
df_train, df_test = create_train_test(df)

In [25]:
df_test_rnn_preprocessed = preprocessor.transform(df_test)

In [26]:
df_test_prophet = prophet_preprocessing(df_test, PROPHET_COLUMN)[-df_test_rnn_preprocessed.shape[0]:]

In [8]:
rnn_model = load_model(ticker)

2022-03-17 15:28:52.493016: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-17 15:28:52.493057: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-17 15:28:52.493075: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (MacBookPro15): /proc/driver/nvidia/version does not exist
2022-03-17 15:28:52.493635: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-17 15:28:54.270214: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond/while' has 1

In [27]:
df_test_prophet

,ds,y
19,2020-03-30,63.702499
20,2020-03-31,63.572498
21,2020-04-01,60.227501
22,2020-04-02,61.232498
23,2020-04-03,60.352501
...,...,...
726,2022-03-07,159.300003
727,2022-03-08,157.440002
728,2022-03-09,162.949997
729,2022-03-10,158.520004


In [28]:
df_test_rnn_preprocessed = np.expand_dims(df_test_rnn_preprocessed, axis=0)

In [29]:
df_test_rnn_preprocessed.shape

(1, 712, 26)

In [30]:
df_test_rnn_preprocessed

array([[[ 2.08278289,  2.14874277, -0.34884981, ...,  0.        ,
          0.        ,  0.        ],
        [ 2.16832185,  2.13961361, -0.18762931, ...,  0.        ,
          0.        ,  0.        ],
        [ 2.00815622,  1.9047153 , -0.30307024, ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 9.03819024,  9.11827555, -0.77388271, ...,  0.        ,
          0.        ,  0.        ],
        [ 8.94807522,  8.80718479, -0.69674786, ...,  0.        ,
          0.        ,  0.        ],
        [ 8.85866384,  8.54103613, -0.74399213, ...,  0.        ,
          0.        ,  0.        ]]])

In [31]:
df_test_rnn_preprocessed_stacked = df_test_rnn_preprocessed[:,df_test_rnn_preprocessed.shape[1] - 15:df_test_rnn_preprocessed.shape[1] - 1,:]
y_test_rnn_preprocessed_stacked = df_test_rnn_preprocessed[0,-1:,1]

In [32]:
df_test_rnn_preprocessed_stacked

array([[[ 9.20434001,  9.25170135, -0.7709983 ,  9.68296823,
          9.61156662,  9.28999668,  0.90190625,  0.        ,
          0.        ,  0.        ,  0.        ,  1.        ,
          0.        ,  0.        ,  0.        ,  1.        ,
          0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 9.20434001,  9.25170135, -0.7709983 ,  9.65823642,
          9.58258872,  9.26532466, -0.03916766,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,
          1.        ,  0.        ,  0.        ,  1.        ,
          0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 9.20434001,  9.25170135, -0.7709983 ,  9.63597271,
          9.55633574,  9.24065265, -0.03916766,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,
          0. 

In [33]:
y_test_rnn_preprocessed_stacked

array([8.54103613])

In [34]:
for observation in range(df_test_rnn_preprocessed.shape[1] - 2, 13, -1):
    df_test_rnn_preprocessed_stacked = np.vstack((df_test_rnn_preprocessed[:,observation - 14:observation,:], df_test_rnn_preprocessed_stacked))
    y_test_rnn_preprocessed_stacked = np.append(df_test_rnn_preprocessed[0,observation,1], y_test_rnn_preprocessed_stacked)

In [35]:
df_test_rnn_preprocessed_stacked.shape

(698, 14, 26)

In [36]:
y_test_rnn_preprocessed_stacked.shape

(698,)

In [37]:
y_test_rnn_preprocessed_stacked

array([ 2.47247436,  2.71474613,  2.66874965,  2.70842625,  2.64013338,
        2.64013338,  2.64013338,  2.5370801 ,  2.38680129,  2.5225089 ,
        2.50372392,  2.64311813,  2.64311813,  2.64311813,  2.64662953,
        2.56604729,  2.72668458,  2.83324873,  2.75020947,  2.75020947,
        2.75020947,  2.82201321,  2.89925924,  2.95315611,  3.00775483,
        3.11993755,  3.11993755,  3.11993755,  3.20561062,  3.14240913,
        3.07639862,  3.10957961,  3.07745193,  3.07745193,  3.07745193,
        3.2047325 ,  3.1727811 ,  3.27969671,  3.23791349,  3.27372776,
        3.27372776,  3.27372776,  3.27372776,  3.23580686,  3.26003361,
        3.2624917 ,  3.2570494 ,  3.2570494 ,  3.2570494 ,  3.32569319,
        3.35185137,  3.38310093,  3.33394451,  3.49510792,  3.49510792,
        3.49510792,  3.52951741,  3.71438145,  3.86975163,  3.57235395,
        3.62326607,  3.62326607,  3.62326607,  3.6968255 ,  3.85640894,
        3.8478067 ,  3.85026479,  3.81497718,  3.81497718,  3.81

In [38]:
df_test_rnn_preprocessed_stacked

array([[[ 2.08278289,  2.14874277, -0.34884981, ...,  0.        ,
          0.        ,  0.        ],
        [ 2.16832185,  2.13961361, -0.18762931, ...,  0.        ,
          0.        ,  0.        ],
        [ 2.00815622,  1.9047153 , -0.30307024, ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 2.3988899 ,  2.38012994, -0.38296286, ...,  0.        ,
          0.        ,  0.        ],
        [ 2.3988899 ,  2.38012994, -0.38296286, ...,  0.        ,
          0.        ,  0.        ],
        [ 2.3988899 ,  2.38012994, -0.38296286, ...,  0.        ,
          0.        ,  0.        ]],

       [[ 2.16832185,  2.13961361, -0.18762931, ...,  0.        ,
          0.        ,  0.        ],
        [ 2.00815622,  1.9047153 , -0.30307024, ...,  0.        ,
          0.        ,  0.        ],
        [ 1.89973642,  1.97528999, -0.36019417, ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 2.3988899 ,  2.38012994, -0.38296286, ...,  

In [40]:
rnn_loss, rnn_mape, rnn_mae = rnn_model.evaluate(df_test_rnn_preprocessed_stacked, y_test_rnn_preprocessed_stacked)

22/22 [==============================] - 0s 6ms/step - loss: 11.1155 - mean_absolute_percentage_error: 39.3006 - mean_absolute_error: 2.9319


In [41]:
rnn_mape

39.30055618286133

In [ ]:
df_test.loc[date.today()] = np.full((1,5), np.nan).tolist()[0]
df_test

In [ ]:
df_test_prophet

In [ ]:
y_test_pred = predict_from_model(ticker, 'prophet', pd.concat([df_test_prophet[['ds']], pd.DataFrame({'ds': [date.today()]})]))
y_test_pred

In [ ]:
np.full((1,5),np.nan).shape

In [ ]:
df_test

In [ ]:
pd.concat([df_test[['Close']], pd.DataFrame()])

In [ ]:
df_test

In [ ]:
pd.concat([pd.concat([df_test[['Close']].loc[date.today()] = np.nan]), pd.DataFrame({'Close_pred_prophet': y_test_pred.values}, index=df_test.index)], axis=1)

In [ ]:
n_days_prediction=1
future = pd.DataFrame({'ds': [date.today()]})
future

In [ ]:
forecast = predict_from_model(ticker, 'prophet', future)
forecast

In [ ]:
mae_prophet = mean_absolute_error(df_test[:n_days_prediction]['y'], forecast['yhat'])
mape_prophet = mean_absolute_percentage_error(df_test[:n_days_prediction]['y'], forecast['yhat'])
print(f'MAE: {mae_prophet}')
pbrint(f'MAPE: {mape_prophet}')

In [ ]:
def predict_from_model(ticker, model_type, X):
    """Predicts from a saved model for a ticker and a model_type
       Input:
            ticker: ticker used by the model
            model_type: type of model used such as prohet or RNN
            X: data
       Ouptut: return a dataframe with predicted values"""
    model = download_model(ticker, model_type)
    if model_type == "prophet":
        y_pred = model['prophet_model'].prophet.predict(X)['yhat']
    else:
        y_pred = model.predict(X)
        
    return y_pred

In [ ]:
cd notebooks

In [ ]:
download_model(ricker, 'prophet')